In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [2]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.54 seconds


In [3]:
## Read Green Taxi Data
import pandas as pd
df = pd.read_csv('green_tripdata_2019-10.csv.gz')

df.dtypes

/var/folders/yk/4b78nv6s13q1wqfgwdvb30wh0000gn/T/ipykernel_45162/3141264447.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('green_tripdata_2019-10.csv.gz')


VendorID                 float64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID               float64
PULocationID               int64
DOLocationID               int64
passenger_count          float64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                float64
improvement_surcharge    float64
total_amount             float64
payment_type             float64
trip_type                float64
congestion_surcharge     float64
dtype: object

In [4]:
df_green = df[[
                'lpep_pickup_datetime', 
                'lpep_dropoff_datetime', 
                'PULocationID', 
                'DOLocationID', 
                'passenger_count', 
                'trip_distance', 
                'tip_amount']]

df_green.dtypes

lpep_pickup_datetime      object
lpep_dropoff_datetime     object
PULocationID               int64
DOLocationID               int64
passenger_count          float64
trip_distance            float64
tip_amount               float64
dtype: object

In [5]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

gt_producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

gt_producer.bootstrap_connected()

True

In [6]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    gt_producer.send(topic_name, value=row_dict)

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 20.45 seconds


In [7]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/anaconda3/envs/de_zc/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/rinlabs/.ivy2/cache
The jars for the packages stored in: /Users/rinlabs/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-05d80930-d9dd-474d-a3c1-492e700409ac;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 266ms :: artifacts dl 9ms


In [8]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [9]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [10]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [11]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

# query = green_stream.writeStream.foreachBatch(peek).start()

In [18]:
query.stop()

In [ ]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/03/24 14:50:54 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/yk/4b78nv6s13q1wqfgwdvb30wh0000gn/T/temporary-4cb8b3c3-ac29-4266-b812-88bad75745d9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/24 14:50:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/03/24 14:50:55 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


In [20]:
popular_destinations = green_stream 

In [21]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/03/24 15:22:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/yk/4b78nv6s13q1wqfgwdvb30wh0000gn/T/temporary-5c6ea15c-ce07-48ea-b3ce-7e9b51f3f745. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/24 15:22:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;
Project [data#21.lpep_pickup_datetime AS lpep_pickup_datetime#23, data#21.lpep_dropoff_datetime AS lpep_dropoff_datetime#24, data#21.PULocationID AS PULocationID#25, data#21.DOLocationID AS DOLocationID#26, data#21.passenger_count AS passenger_count#27, data#21.trip_distance AS trip_distance#28, data#21.tip_amount AS tip_amount#29]
+- Project [from_json(StructField(lpep_pickup_datetime,StringType,true), StructField(lpep_dropoff_datetime,StringType,true), StructField(PULocationID,IntegerType,true), StructField(DOLocationID,IntegerType,true), StructField(passenger_count,DoubleType,true), StructField(trip_distance,DoubleType,true), StructField(tip_amount,DoubleType,true), cast(value#8 as string), Some(Asia/Bangkok)) AS data#21]
   +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@4b4eeb48, kafka, org.apache.spark.sql.kafka010.KafkaSourcePr